### Script translation text(en) to malagasy(mg)

In [ ]:
import subprocess
import sys
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import torch
from google.colab import drive
from tqdm.auto import tqdm
import wandb

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Définir les arguments pour le script
parser = argparse.ArgumentParser(description='Translate a portion of a CSV file using MADLAD-400-7B-MT-BT.')
parser.add_argument('--start_index', type=int, default=0, help='Index de la première ligne à traiter (inclusive).')
parser.add_argument('--end_index', type=int, default=10000, help='Index de la dernière ligne à traiter (exclusive).')
parser.add_argument('--batch_size', type=int, default=16, help='Taille du lot pour la traduction.')
args = parser.parse_args()

start_index = args.start_index
end_index = args.end_index
batch_size = args.batch_size

# Vérifier si un GPU est disponible et le définir comme device, sinon utiliser le CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilisation de l'appareil : {device}")

# modèle MADLAD-400-7B-MT-BT
model_name = "google/madlad400-7b-mt-bt"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto').to(device)

# Monter Google Drive pour le chargement et les sauvegardes des données
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/hate_speech/data_combined.csv'
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Erreur: Le fichier '{file_path}' n'a pas été trouvé.")
    exit()

# Vérifier que la colonne 'text' existe
if 'text' not in df.columns:
    print("Erreur: La colonne 'text' n'existe pas dans le fichier CSV.")
    exit()

# Sélectionner la portion du DataFrame à traiter
df_subset = df[start_index:end_index]
print(f"Traitement des lignes de {start_index} à {end_index - 1} par lots de {batch_size}.")

def translate_batch(texts):
    input_texts = [f"<2mg> {text}" for text in texts]  # Ajouter le préfixe de langue
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        generated_tokens = model.generate(
            **inputs,
            max_length=512
        )
    translated_texts = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated_texts

translations = [''] * len(df_subset)  # Pré-allouer la liste des traductions

# Appliquer la traduction par lots avec une barre de progression
for i in tqdm(range(0, len(df_subset), batch_size), desc="Traduction par lots"):
    batch = df_subset['text'][i:i + batch_size].tolist()
    translated_batch = translate_batch(batch)
    translations[i:i + batch_size] = translated_batch

# Ajouter la liste des traductions comme une nouvelle colonne au sous-ensemble
df_subset['translated'] = translations

# Initialiser WandB
wandb.init(project="hate-speech-translated-malagasy")
# Créer un tableau WandB à partir du sous-ensemble traduit (contenant 'text', 'label' et 'translated')
wandb_table = wandb.Table(data=df_subset[['text', 'label', 'translated']], columns=["text", "label", "translated"])

# Enregistrer le tableau dans WandB
wandb.log({"translations": wandb_table})

# Fermer la run WandB
wandb.finish()

Utilisation de l'appareil : cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Traitement des lignes de 400 à 599 par lots de 16.


<ipython-input-3-541c40815379>:20: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Traduction par lots:   0%|          | 0/13 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-3-541c40815379>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['translated'] = translations
wandb: Currently logged in as: colab-rianabenandriarinaivo (colab-rianabenandriarinaivo-riana-ben) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
